# Toronto Neighborhoods
Author:    Guy Shavit  
### Part 1: scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

#### Importing packages

In [1]:
import pandas as pd

#### Get the Wikipedia page and read into a Dataframe

In [2]:
!wget -O List_of_postal_codes_of_Canada:_M https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
df = pd.read_html('List_of_postal_codes_of_Canada:_M')

--2021-02-12 20:01:02--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54740 (53K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.46K  --.-KB/s    in 0.003s  

2021-02-12 20:01:03 (17.5 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54740/54740]



#### Take the first table from the Wikipedia page into a Dataframe, Drop any line with Boroguh "Not assigned", reset the index and set the Neighbourhood to the Borough name for any Neighbourhood that is "Not assigned"

In [3]:
PostCode = df[0]
PostCode.drop(PostCode.loc[PostCode['Borough']=="Not assigned"].index, inplace=True)
PostCode.reset_index(drop=True, inplace=True)
PostCode.loc[PostCode.Neighbourhood == "Not assigned", "Neighbourhood"] = PostCode.Borough
PostCode.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Get the table shape

In [4]:
PostCode.shape

(103, 3)

### Part 2:  Add latitude and the longitude coordinates of each neighborhood

#### Importing packages

In [5]:
!pip install geocoder
import geocoder


#### Create a list of latitudes and longitudes for each postal code 
#### lookup the latitude and longitude ia geocoder and add to the list.  
#### At the end add the lists to the dataframe

In [6]:
latitude = []
longitude = []
for index, PC in PostCode.iterrows():
    # initialize lat_lng variable to None
    lat_lng_coords = None
    postal_code = PC['Postal Code']
    #loop until you get the coordinates
    while(lat_lng_coords is None):
          g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
          lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
PostCode['Latitude'] = latitude
PostCode['Longitude'] = longitude

In [7]:
PostCode.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


### Part 3:  Explore and cluster the neighborhoods in Toronto

#### Importing packages

In [8]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



#### Use geopy library to get the latitude and longitude values of Toronto.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(PostCode['Latitude'], PostCode['Longitude'], PostCode['Borough'], PostCode['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'ULAMQLZ3O2MJYTHU2QTK0QYRUZ3YP3ZNGOSFZ45PG2R4TQ0I' 
CLIENT_SECRET = 'Z5LBQMM1WEPQSJJWIQ3ALKHCVFLD4VR1DTSX1MCE0ZURWW3O' 
ACCESS_TOKEN = 'MZI1APQNRECXU5PRJZMY4BLZUHRGYZEZ24SU3NEYXMKWGV03' 

VERSION = '20200212' # Foursquare API version
LIMIT = 100 

#### function to get top 100 venus within 500m of a location

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    radius=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### run the function to get nearby venues for neighbourhood in Toronto

In [13]:
toronto_venues = getNearbyVenues(names=PostCode['Neighbourhood'],
                                   latitudes=PostCode['Latitude'],
                                   longitudes=PostCode['Longitude']
                                  )

#### size and result dataframe

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(2380, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
4,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop


#### how many venues were returned for each neighborhood?

In [15]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,14,14,14,14,14,14
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,5,5,5,5,5,5
Woodbine Heights,14,14,14,14,14,14


#### How many of each catagory in each Neighbourhood?

In [16]:
toronto_venues.groupby(['Neighbourhood','Venue Category']).count()

Neighbourhood Latitude  \
Neighbourhood            Venue Category                               
Agincourt                Badminton Court                          1   
                         Bakery                                   1   
                         Bubble Tea Shop                          1   
                         Chinese Restaurant                       1   
                         Department Store                         1   
...                                                             ...   
York Mills West          Coffee Shop                              1   
                         Convenience Store                        1   
                         Park                                     1   
                         Speakeasy                                1   
York Mills, Silver Hills Park                                     1   

                                             Neighbourhood Longitude  Venue  \
Neighbourhood            Venue Category                                       
Agincourt                Badminton Court                           1      1   
                         Bakery                                    1      1   
                         Bubble Tea Shop                           1      1   
                         Chinese Restaurant                        1      1   
                         Department Store                          1      1   
...                                                              ...    ...   
York Mills West          Coffee Shop                               1      1   
                         Convenience Store                         1      1   
                         Park                                      1      1   
                         Speakeasy                                 1      1   
York Mills, Silver Hills Park                                      1      1   

                                             Venue Latitude  Venue Longitude  
Neighbourhood            Venue Category                                       
Agincourt                Badminton Court                  1                1  
                         Bakery                           1                1  
                         Bubble Tea Shop                  1                1  
                         Chinese Restaurant               1                1  
                         Department Store                 1                1  
...                                                     ...              ...  
York Mills West          Coffee Shop                      1                1  
                         Convenience Store                1                1  
                         Park                             1                1  
                         Speakeasy                        1                1  
York Mills, Silver Hills Park                             1                1  

[1678 rows x 5 columns]

#### Which Neighbourhood has the most Chinese restaurants?

In [17]:
toronto_venues[toronto_venues["Venue Category"] =='Chinese Restaurant'].groupby('Neighbourhood').count().sort_values(['Venue Category'], ascending=False).head(1)

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Garden District, Ryerson",2,2,2,2,2,2


### Analyze each Neighbourhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [19]:
toronto_onehot.shape

(2380, 265)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
94,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
95,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
96,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


#### Confirm the new size

In [21]:
toronto_grouped.shape

(98, 265)

#### Let's print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0       Shopping Mall  0.07
1    Sushi Restaurant  0.07
2  Chinese Restaurant  0.07
3     Badminton Court  0.07
4         Supermarket  0.07


----Alderwood, Long Branch----
                   venue  freq
0      Convenience Store  0.25
1                    Gym  0.25
2                    Pub  0.25
3  Performing Arts Venue  0.25
4      Accessories Store  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0       Home Service   1.0
1  Accessories Store   0.0
2             Museum   0.0
3          Nightclub   0.0
4          Newsagent   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0.25
2                        Park  0.25
3           Accessories Store  0.00
4                 Music Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.10
1      Sandwich Place  0.10
2    

#### put that into a panada dataframe

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Hong Kong Restaurant,Grocery Store,Chinese Restaurant,Supermarket,Bubble Tea Shop,Bakery,Badminton Court,Newsagent,Discount Store,Department Store
1,"Alderwood, Long Branch",Convenience Store,Performing Arts Venue,Pub,Gym,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
2,"Bathurst Manor, Wilson Heights, Downsview North",Home Service,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
3,Bayview Village,Trail,Construction & Landscaping,Park,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
4,"Bedford Park, Lawrence Manor East",Thai Restaurant,Italian Restaurant,Sandwich Place,Coffee Shop,Comfort Food Restaurant,Indian Restaurant,Juice Bar,Pub,Liquor Store,Restaurant


### Cluster the Neighbourhoods
#### Run _k_-means to cluster the neighborhood into 5 clusters.

In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 4, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = PostCode
toronto_merged = neighbourhoods_venues_sorted

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(PostCode.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.groupby('Cluster Labels').head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,2,Agincourt,Hong Kong Restaurant,Grocery Store,Chinese Restaurant,Supermarket,Bubble Tea Shop,Bakery,Badminton Court,Newsagent,Discount Store,Department Store,M1S,Scarborough,43.79452,-79.26708
1,0,"Alderwood, Long Branch",Convenience Store,Performing Arts Venue,Pub,Gym,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,M8W,Etobicoke,43.60124,-79.53879
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Home Service,Yoga Studio,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,M3H,North York,43.75788,-79.44847
3,1,Bayview Village,Trail,Construction & Landscaping,Park,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,M2K,North York,43.78112,-79.38060
4,2,"Bedford Park, Lawrence Manor East",Thai Restaurant,Italian Restaurant,Sandwich Place,Coffee Shop,Comfort Food Restaurant,Indian Restaurant,Juice Bar,Pub,Liquor Store,Restaurant,M5M,North York,43.73545,-79.41916
5,2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Cheese Shop,Breakfast Spot,Farmers Market,Restaurant,Bakery,Optical Shop,M5E,Downtown Toronto,43.64536,-79.37306
6,2,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Yoga Studio,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,M1N,Scarborough,43.69510,-79.26466
7,2,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bar,Café,Restaurant,Gift Shop,Sandwich Place,Nightclub,Lounge,Japanese Restaurant,Italian Restaurant,M6K,West Toronto,43.63941,-79.42676
26,1,"East Toronto, Broadview North (Old East York)",Intersection,Park,Music Venue,Film Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,M4J,East York,43.68811,-79.33418
27,1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Carpet Store,Electronics Store,Fish & Chips Shop,Park,College Rec Center,Grocery Store,Shopping Mall,Farm,Ethiopian Restaurant,Event Space,M9C,Etobicoke,43.64857,-79.57825


#### Finally, let's visualize the resulting clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
1,"Alderwood, Long Branch",Gym,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,M8W,Etobicoke,43.60124,-79.53879


#### Cluster 2

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
3,Bayview Village,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,M2K,North York,43.78112,-79.38060
26,"East Toronto, Broadview North (Old East York)",Film Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio,M4J,East York,43.68811,-79.33418
27,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Park,College Rec Center,Grocery Store,Shopping Mall,Farm,Ethiopian Restaurant,Event Space,M9C,Etobicoke,43.64857,-79.57825
34,"Guildwood, Morningside, West Hill",Tea Room,Fish & Chips Shop,Film Studio,Field,Fast Food Restaurant,Fish Market,Dumpling Restaurant,M1E,Scarborough,43.76575,-79.17470
36,"High Park, The Junction South",Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,M6P,West Toronto,43.65973,-79.46281
37,Hillcrest Village,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio,Dumpling Restaurant,M2H,North York,43.80225,-79.35558
39,"Humberlea, Emery",Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,M9M,North York,43.73367,-79.53769
40,Humewood-Cedarvale,Grocery Store,Trail,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,M6C,York,43.69211,-79.43036
53,"Moore Park, Summerhill East",Tennis Court,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,M4T,Central Toronto,43.69048,-79.38318
54,"New Toronto, Mimico South, Humber Bay Shores",Skating Rink,Convenience Store,Tennis Court,Fish & Chips Shop,Film Studio,Field,Fish Market,M8V,Etobicoke,43.60987,-79.49809


#### Cluster 3

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,Agincourt,Supermarket,Bubble Tea Shop,Bakery,Badminton Court,Newsagent,Discount Store,Department Store,M1S,Scarborough,43.79452,-79.26708
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Comfort Food Restaurant,Indian Restaurant,Juice Bar,Pub,Liquor Store,Restaurant,M5M,North York,43.73545,-79.41916
5,Berczy Park,Beer Bar,Cheese Shop,Breakfast Spot,Farmers Market,Restaurant,Bakery,Optical Shop,M5E,Downtown Toronto,43.64536,-79.37306
6,"Birch Cliff, Cliffside West",College Stadium,Yoga Studio,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,M1N,Scarborough,43.69510,-79.26466
7,"Brockton, Parkdale Village, Exhibition Place",Restaurant,Gift Shop,Sandwich Place,Nightclub,Lounge,Japanese Restaurant,Italian Restaurant,M6K,West Toronto,43.63941,-79.42676
...,...,...,...,...,...,...,...,...,...,...,...,...
89,Weston,Grocery Store,Diner,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,M9N,York,43.70507,-79.51804
90,"Wexford, Maryvale",Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Eastern European Restaurant,M1R,Scarborough,43.75075,-79.30054
91,"Willowdale, Newtonbrook",Korean Restaurant,Japanese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Supermarket,Bakery,M2M,North York,43.79182,-79.41372
92,"Willowdale, Willowdale East",Steakhouse,Liquor Store,Lounge,Shopping Mall,Middle Eastern Restaurant,Café,Movie Theater,M2N,North York,43.76774,-79.40728


#### Cluster 4

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
82,"The Kingsway, Montgomery Road, Old Mill North",Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dumpling Restaurant,M8X,Etobicoke,43.65319,-79.51113
86,Victoria Village,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Dumpling Restaurant,M4A,North York,43.73057,-79.31306
97,"York Mills, Silver Hills",Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Dumpling Restaurant,M2L,North York,43.75698,-79.38060


#### Cluster 5

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
1,"Alderwood, Long Branch",Gym,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,M8W,Etobicoke,43.60124,-79.53879
